## Introducción
### ¿Cuál es el automóvil más económico y confiable que puedo comprar, considerando que lo necesito para uso diario en la ciudad?

**cliente 1:**
Estoy buscando un auto principalmente para movilizarme en la ciudad. No me interesa que sea de lujo, pero sí que sea económico en precio (entre 8,000 y 15,000 USD) y en consumo de combustible. Prefiero que tenga transmisión automática, porque me resulta más cómoda en el tráfico. No me importa mucho la marca, pero quiero que el auto tenga un kilometraje bajo y que no sea demasiado antiguo (idealmente de los últimos 10 años). El presupuesto que tengo es limitado, así que necesito que el vehículo esté en buen estado y que sea accesible en precio. 

## Librerías y ajuste de parámetros

In [2]:
import sys
sys.path.append('..')

from src.utils import configurar_graficas, track_data_reduction
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

configurar_graficas()

## Carga de datos

Se cargarán los datos de 'carros_parte2.csv' y se inspeccionarán inicialmente para entender su estructura y contenido.

In [3]:
data = pd.read_csv('../Data/carros_parte2.csv',  parse_dates=['year_produced'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16747 entries, 0 to 16746
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   manufacturer_name  16747 non-null  object        
 1   model_name         16747 non-null  object        
 2   transmission       16747 non-null  object        
 3   color              16747 non-null  object        
 4   odometer_value     16747 non-null  int64         
 5   year_produced      16747 non-null  datetime64[ns]
 6   engine_fuel        16747 non-null  object        
 7   engine_has_gas     16747 non-null  bool          
 8   engine_type        16747 non-null  object        
 9   engine_capacity    16747 non-null  float64       
 10  body_type          16747 non-null  object        
 11  has_warranty       16747 non-null  bool          
 12  state              16747 non-null  object        
 13  drivetrain         16747 non-null  object        
 14  price_